In [1]:
%cd ~/austin_big_vision
import jax
import jax.numpy as jnp
import ml_collections

/home/austinwang/austin_big_vision


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# CapPa ViT

In [5]:
import big_vision.models.vit as model_mod
model_cfg = ml_collections.ConfigDict()
model_cfg.num_classes = None
model_cfg.patch_size = (16, 16)
model_cfg.width = 768
model_cfg.depth = 12
model_cfg.mlp_dim = None  # Defaults to 4x input dim
model_cfg.num_heads = 12
model_cfg.posemb = "learn"  # Can also be "sincos2d"
model_cfg.rep_size = False
model_cfg.dropout = 0.0
model_cfg.pool_type = "map"  # Can also be "map" or "tok"
model_cfg.head_zeroinit = True
model_cfg.mask = None
model_cfg.normalize_qk = False
model_cfg.scan = True
model_cfg.remat_policy = "nothing_saveable"
model_cfg.dtype_mm = "bfloat16"

model = model_mod.Model(**model_cfg)

cappa_ckpt = 'gs://us-central2-storage/tensorflow_datasets/cappa_bs16384_warm0.02_lr1e-3_wd1e-4_bf16_b2-0.95_6lyr_06-15_2102/checkpoint.bv-000183105'
cappa_path = cappa_ckpt+':encoder'

init_params = None
params = model_mod.load(init_params, cappa_path, model_cfg)
jax.tree_map(jnp.shape, params)

/tmp/ipykernel_1027117/63146067.py:27: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  jax.tree_map(jnp.shape, params)


{'Transformer': {'encoder_norm': {'bias': (768,), 'scale': (768,)},
  'encoderblock': {'LayerNorm_0': {'bias': (12, 768), 'scale': (12, 768)},
   'LayerNorm_1': {'bias': (12, 768), 'scale': (12, 768)},
   'MlpBlock_0': {'Dense_0': {'bias': (12, 3072), 'kernel': (12, 768, 3072)},
    'Dense_1': {'bias': (12, 768), 'kernel': (12, 3072, 768)}},
   'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 12, 64),
     'kernel': (12, 768, 12, 64)},
    'key_ln': {'scale': (12, 64)},
    'out': {'bias': (12, 768), 'kernel': (12, 12, 64, 768)},
    'query': {'bias': (12, 12, 64), 'kernel': (12, 768, 12, 64)},
    'query_ln': {'scale': (12, 64)},
    'value': {'bias': (12, 12, 64), 'kernel': (12, 768, 12, 64)}}}},
 'embedding': {'bias': (768,), 'kernel': (16, 16, 3, 768)},
 'pos_embedding': (1, 196, 768)}

# SigLIP ViT

In [6]:
siglip_ckpt = 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/siglip_replication_pod_04-11_2247/checkpoint.bv-000183105'
siglip_path = siglip_ckpt+':img'

init_params = None
params = model_mod.load(init_params, siglip_path, model_cfg)
jax.tree_map(jnp.shape, params)

/tmp/ipykernel_1027117/3292360820.py:6: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  jax.tree_map(jnp.shape, params)


{'MAPHead_0': {'LayerNorm_0': {'bias': (768,), 'scale': (768,)},
  'MlpBlock_0': {'Dense_0': {'bias': (3072,), 'kernel': (768, 3072)},
   'Dense_1': {'bias': (768,), 'kernel': (3072, 768)}},
  'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 64),
    'kernel': (768, 12, 64)},
   'out': {'bias': (768,), 'kernel': (12, 64, 768)},
   'query': {'bias': (12, 64), 'kernel': (768, 12, 64)},
   'value': {'bias': (12, 64), 'kernel': (768, 12, 64)}},
  'probe': (1, 1, 768)},
 'Transformer': {'encoder_norm': {'bias': (768,), 'scale': (768,)},
  'encoderblock': {'LayerNorm_0': {'bias': (12, 768), 'scale': (12, 768)},
   'LayerNorm_1': {'bias': (12, 768), 'scale': (12, 768)},
   'MlpBlock_0': {'Dense_0': {'bias': (12, 3072), 'kernel': (12, 768, 3072)},
    'Dense_1': {'bias': (12, 768), 'kernel': (12, 3072, 768)}},
   'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 12, 64),
     'kernel': (12, 768, 12, 64)},
    'out': {'bias': (12, 768), 'kernel': (12, 12, 64, 768)},
    'query': {'